In [7]:
import chess
import random
from stockfish import Stockfish
stockfish = Stockfish(path="stockfish_15.1/src/evaluate.cpp")
# calculates pawn until it becomes queen
def calculatePawn(piece, position):
    print("Calculated")
    #bestMove(piece,position)

def isCheckmate(board):
    if board.is_checkmate():
        return True
    else:
        return False

##Takes chess piece type and position and evals best move
def bestMove(board):
    board_original = board.copy()
    #Converts index into square name like b4
    #print(chess.square_name(chess.BLACK))
    #print(chess.square_name(chess.WHITE))
    fen = str(board.fen())
    print(fen)

    #List of legal moves for whos turn it is
    legal_moves = list(board.legal_moves)
    black_king= board.king(chess.BLACK)
    white_king= board.king(chess.WHITE)
    king_distance = chess.square_distance(white_king, black_king)
    ##Checks for whos turn it is
    if board.turn:
        print("Whites move")
        if(fen.__contains__("R")):
            ##If theres a move for checkmate. 
            for move in legal_moves:
                # Make the move on a copy of the board
                board_copy = board.copy()
                board_copy.push(move)
                if isCheckmate(board_copy):
                    board.push(move)
                    break
            if board_original == board:

                ##if there is no way to checkmate limit black king to only have 2 possible moves
                print("No way to checkmate in this position")
                for move in legal_moves:
                # Make the move on a copy of the board
                    board_copy = board.copy()
                    board_copy.push(move)

                    black_king2= board_copy.king(chess.BLACK)
                    white_king2= board_copy.king(chess.WHITE)
                    white_rook2 = next(iter(board_copy.pieces(chess.ROOK, chess.WHITE)))
                    king_distance_2 = chess.square_distance(white_king2, black_king2)
                    rook_distance = chess.square_distance(black_king, white_rook2)
                    if king_distance_2 <= 2:
                        if board_copy.legal_moves.count() <= 2 and king_distance >=2 and rook_distance > 1 and not board_copy.is_check():
                            board.push(move)
                            break
                    elif king_distance > king_distance_2:
                        board.push(move)
                        break
                    
                print(board)
                threePieceGen(board)
            else:
                print(board)
                threePieceGen(board)
            
            

        elif(fen.__contains__("Q")):
            print("I am the Queen")
            white_queen = next(iter(board.pieces(chess.QUEEN, chess.WHITE)))
            print(chess.square_name(white_queen))
            ##Krishna or Jonathan figure out how to move a 3 piece queen game
        else:
            #WhiteKing is alone
            print("I am losing")
            

    ## Blacks move
    else:
        print("Blacks move")
        if(fen.__contains__("r")):
            print("I am the Rook")
            ##Krishna or Jonathan figure out how to move a 3 piece rook game
        elif(fen.__contains__("q")):
            print("I am the Queen")
            white_queen = next(iter(board.pieces(chess.QUEEN, chess.WHITE)))
            print(chess.square_name(white_queen))
            ##Krishna or Jonathan figure out how to move a 3 piece queen game
        else:
            #BlackKing is alone
            print("I am losing")
            legal_moves = list(board.legal_moves)
            move = random.choice(legal_moves)
            board.push(move)
            print(board)
            threePieceGen(board)
            

            


def twoPieceGen(board):
    print("results will always equal draw ")
    print(board.fen())
    #Krishna or Jonathon Add code to import to DynamoDB. Import FEN and That it will be a draw

def threePieceGen(board):

    if len(board.piece_map()) ==2:
        twoPieceGen(board)
    elif isCheckmate(board):
        results = board.result()
        print(board.fen())
        if results == "1-0":
            print(board.fen())
            print("White wins!")
        elif results == "0-1":
            print("Black wins!")
        #Store FEN and WL into DynamoDB
        print(board)
    else:
        bestMove(board)


## ----------Additional block for checking each sqaure and legal moves the piece can make
   # for square in chess.SQUARES:
    #    piece = board.piece_at(square)
     #   if piece is not None:
      #      print("\n")
       #     moves = board.legal_moves
        #    bestMove(piece, chess.square_name(square))
            #print(f"{piece} on square {chess.square_name(square)} can move to:")
            #for move in moves:
                #print(f"- {chess.Move.uci(move)}")

## --------------------------------------Start READING HERE -----------------------------------------
#Diffrent board states to test
#KRvK
#board = chess.Board("5k2/8/8/4R1K1/8/8/8/8 w - - 0 1")
#Move king closer to other king
board = chess.Board("7k/8/2R5/6K1/8/8/8/8 w - - 0 1")
# One move to checkmate
#board = chess.Board("6k1/8/4R1K1/8/8/8/8/8 w - - 0 1")

#KPvK
#board = chess.Board("8/8/P1K3k1/8/8/8/8/8 w - - 0 1")

#KQvK
#board = chess.Board("8/2Q5/6k1/8/3K4/8/8/8 w - - 0 1")

#KvK
#board = chess.Board("4k3/8/8/4K3/8/8/8/8 b - - 0 1")

#KBvK
#board = chess.Board("8/5k2/8/2B1K3/8/8/8/8 w - - 0 1")

#KNvK
#board = chess.Board("8/2n2k2/8/4K3/8/8/8/8 b - - 0 1")

#winning board state
#board = chess.Board("4R1k1/8/6K1/8/8/8/8/8 b - - 0 1")

print("Total number of pieces on the board:", len(board.piece_map()))
print(board)
stockfish.get_best_move()

#Two piece generator
if len(board.piece_map()) ==2:
   twoPieceGen(board)
   

#Three piece generator
if len(board.piece_map()) ==3:
    fenBoard = str(board.fen())
    print(fenBoard)
    if fenBoard.count("b") > 1 or fenBoard.count("B") >= 1 :
        print("Bishop and king cannot checkmate. Therefore draw")
        print("results will always equal draw ")
        #Krishna or Jonathon Add code to import to DynamoDB. Import FEN and That it will be a draw
    elif fenBoard.__contains__('n') or fenBoard.__contains__('N'):
        print("Knight and king cannot checkmate. Therefore draw")
    
    elif fenBoard.__contains__('p') or fenBoard.__contains__('P'):
        print("Theres a pawn therfore we need more calculations")
        #CalculatePawn()

    else:
        threePieceGen(board)
    


# Print the FEN string and pieces
pieces = board.piece_map()
fen = board.fen()
#print("FEN: ", fen)
#print("Pieces: ", pieces)





OSError: [WinError 193] %1 is not a valid Win32 application